# 🛍️ | Cora-For-Zava: Model Selection 

Welcome! This notebook will walk you through evaluating multiple AI models using a standardized test dataset and the Azure AI Evaluation SDK.

## 🛒 Our Zava Scenario

**Cora** is a customer service chatbot for **Zava** - a fictitious retailer of home improvement goods for DIY enthusiasts. To ensure Cora provides the best customer experience, you need to select the right foundation model. With multiple Azure OpenAI models available (GPT-4o, GPT-4o-mini, GPT-4), you need to evaluate which model delivers the best balance of quality, safety, and performance for your retail use case.

## 🎯 What You'll Build

By the end of this notebook, you'll have:
- ✅ Configured multiple Azure OpenAI models for comparison
- ✅ Loaded standardized test datasets for evaluation
- ✅ Run evaluations across models using built-in evaluators
- ✅ Analyzed performance metrics (quality, safety, latency)
- ✅ Compared model results to make informed selection decisions

## 💡 What You'll Learn

- How to configure multiple models for evaluation
- How to load test datasets for evaluation
- How to run evaluations with built-in evaluators
- How to analyze and compare model performance
- How to use Azure AI Foundry model leaderboards

> **Note**: This demonstrates pre-production evaluation, which is essential before deploying AI applications.

Ready to compare models? Let's get started! 🚀

---

## Step 1: Verify Environment Variables

The following environment variables should already be configured in your `.env` file from the earlier setup steps:

- **AZURE_OPENAI_API_KEY**: Your Azure OpenAI API key
- **AZURE_OPENAI_ENDPOINT**: Your Azure OpenAI service endpoint
- **AZURE_OPENAI_API_VERSION**: The API version to use
- **AZURE_SUBSCRIPTION_ID**: Your Azure subscription ID
- **AZURE_RESOURCE_GROUP**: Your Azure resource group name
- **AZURE_AI_PROJECT_NAME**: Your Azure AI Foundry project name

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
# Use override=True to reload any changes made to .env
load_dotenv(override=True)

# Verify all required Azure service credentials are available
required_vars = [
    "AZURE_OPENAI_API_KEY",
    "AZURE_OPENAI_ENDPOINT", 
    "AZURE_OPENAI_API_VERSION",
    "AZURE_SUBSCRIPTION_ID",
    "AZURE_RESOURCE_GROUP",
    "AZURE_AI_PROJECT_NAME",
    "AZURE_AI_FOUNDRY_NAME"
]

missing_vars = [var for var in required_vars if not os.environ.get(var)]

if missing_vars:
    raise EnvironmentError(f"❌ Missing environment variables: {', '.join(missing_vars)}")

print("✅ All environment variables configured!")


## Step 2: Define Models to Evaluate

Configure the array of model deployment names you want to evaluate. You can add or remove models from this list based on what's deployed in your Azure OpenAI resource.

> **Tip**: Use [Azure AI Foundry Model Leaderboards](https://learn.microsoft.com/azure/ai-foundry/how-to/benchmark-model-in-catalog) to compare models on quality, safety, cost, and performance before deploying them.

In [ ]:
# Define the models to evaluate
# Add or remove model deployment names as needed
models_to_evaluate = [
    "gpt-4o-mini",
    "gpt-4o",
    "gpt-4.1"
]

print(f"✅ Configured {len(models_to_evaluate)} models for evaluation")

## 💡 Model Selection with Leaderboards

Before or after running custom evaluations, you can use Azure AI Foundry Model Leaderboards to help select the best models:

**How to Access Leaderboards:**
1. Go to [Azure AI Foundry portal](https://ai.azure.com)
2. Select **Model catalog** from the left pane
3. Click **Browse leaderboards** in the Model leaderboards section

**What You Can Compare:**
- **Quality Leaderboard**: Models ranked by accuracy on reasoning, Q&A, coding, and math tasks
- **Safety Leaderboard**: Models ranked by resistance to harmful content
- **Cost Leaderboard**: Models ranked by cost-effectiveness
- **Performance Leaderboard**: Models ranked by throughput and latency
- **Trade-off Charts**: Quality vs. Cost, Quality vs. Safety, Quality vs. Throughput
- **Scenario-Specific**: Find models best for your use case (chatbots, code generation, etc.)

This can help you narrow down which models to include in your `models_to_evaluate` list!

---

## Step 3: Load Test Dataset

Load the evaluation dataset with test queries and expected responses. This will be used as the test input for all models.

In [ ]:
import json
import pandas as pd

# Load the evaluation dataset
dataset_path = "22-evaluate-models.jsonl"
test_data = []

with open(dataset_path, "r") as f:
    for line in f:
        test_data.append(json.loads(line))

print(f"✅ Loaded {len(test_data)} test examples from {dataset_path}\n")

# Display as DataFrame for easy viewing
df_test_data = pd.DataFrame(test_data)
print("📊 Test Dataset Preview:")
display(df_test_data)


## Step 4: Configure Azure AI Project

Set up the Azure AI project connection for running evaluations.

In [ ]:
from azure.ai.evaluation import AzureOpenAIModelConfiguration
from azure.identity import DefaultAzureCredential

# Create Azure AI project configuration
subscription_id = os.environ.get("AZURE_SUBSCRIPTION_ID")
resource_group_name = os.environ.get("AZURE_RESOURCE_GROUP")
project_name = os.environ.get("AZURE_AI_PROJECT_NAME")
azure_ai_foundry_name = os.environ.get("AZURE_AI_FOUNDRY_NAME")

# Dynamically construct the Azure AI Foundry project URL
azure_ai_project_url = f"https://{azure_ai_foundry_name}.services.ai.azure.com/api/projects/{project_name}"

# Initialize and verify credential
try:
    credential = DefaultAzureCredential()
    # Try to get a token to verify authentication
    token = credential.get_token("https://management.azure.com/.default")
    print("✅ Azure credentials verified successfully!")
except Exception as e:
    print("❌ Azure credentials not found or expired!")
    print("Please run 'az login' in the terminal to authenticate with Azure.")
    raise

print(f"✅ Azure AI Project configured")


## Step 5: Create Model Configurations

Create model configuration objects for each model we want to evaluate.

In [ ]:
# Create model configurations for all models
model_configs = {}

for model_name in models_to_evaluate:
    model_configs[model_name] = AzureOpenAIModelConfiguration(
        azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
        azure_deployment=model_name,
        api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
        api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
    )
    
print(f"✅ Created configurations for {len(model_configs)} models")

## Step 6: Define Target Function

Create a function that takes a query and returns a response from a specific model. This will be used by the evaluators.

In [ ]:
from openai import AzureOpenAI

def create_target_function(model_name):
    """Create a target function for a specific model"""
    
    def target_function(query: str, ground_truth: str = None, response: str = None) -> dict:
        """Generate response from the model"""
        client = AzureOpenAI(
            api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
            api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
            azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT")
        )
        
        # Call the model with the query
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": "You are a helpful assistant that answers questions accurately and concisely."},
                {"role": "user", "content": query}
            ],
            temperature=0.7,
            max_tokens=800
        )
        
        return {
            "response": response.choices[0].message.content
        }
    
    return target_function

print("✅ Target function factory created")

## Step 7: Configure Evaluators

Set up the evaluators we'll use to assess model performance. We'll use [built-in evaluators](https://learn.microsoft.com/azure/ai-foundry/concepts/observability#what-are-evaluators) for quality and safety metrics.

**Quality Evaluators** (AI-assisted):
- **Relevance**: Evaluates how pertinent responses are to the given questions (scale 1-5)
- **Coherence**: Evaluates how well the output flows smoothly and reads naturally (scale 1-5)
- **Fluency**: Evaluates language proficiency and grammatical correctness (scale 1-5)

**Safety Evaluators** (Content safety):
- **Violence**: Detects violent content in responses (scale 0-7, lower is safer)
- **Hate/Unfairness**: Detects hateful or unfair content (scale 0-7, lower is safer)
- **Self-Harm**: Detects self-harm related content (scale 0-7, lower is safer)
- **Sexual**: Detects sexual content (scale 0-7, lower is safer)

> **Note**: We're using Relevance, Coherence, and Fluency evaluators which don't require context or ground truth. Groundedness evaluator has been removed as it requires additional context that our simple dataset doesn't provide.

> Learn more about [evaluation metrics](https://learn.microsoft.com/azure/machine-learning/prompt-flow/concept-model-monitoring-generative-ai-evaluation-metrics) and their use cases.

In [ ]:
from azure.ai.evaluation import (
    GroundednessEvaluator,
    RelevanceEvaluator,
    CoherenceEvaluator,
    FluencyEvaluator,
    ViolenceEvaluator,
    HateUnfairnessEvaluator,
    SelfHarmEvaluator,
    SexualEvaluator
)

# Create a judge model configuration for evaluators
judge_model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    azure_deployment="gpt-4o",  # Use a capable model as judge
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
)

# Initialize quality evaluators
groundedness_eval = GroundednessEvaluator(model_config=judge_model_config)
relevance_eval = RelevanceEvaluator(model_config=judge_model_config)
coherence_eval = CoherenceEvaluator(model_config=judge_model_config)
fluency_eval = FluencyEvaluator(model_config=judge_model_config)

# Initialize safety evaluators (using azure_ai_project_url instead of dictionary)
violence_eval = ViolenceEvaluator(azure_ai_project=azure_ai_project_url, credential=credential)
hate_unfairness_eval = HateUnfairnessEvaluator(azure_ai_project=azure_ai_project_url, credential=credential)
self_harm_eval = SelfHarmEvaluator(azure_ai_project=azure_ai_project_url, credential=credential)
sexual_eval = SexualEvaluator(azure_ai_project=azure_ai_project_url, credential=credential)

print("✅ Evaluators configured:")
print("   Quality: Groundedness, Relevance, Coherence, Fluency")
print("   Safety: Violence, Hate/Unfairness, Self-Harm, Sexual")


In [ ]:
# Test with a single model and single prompt
import sys
from datetime import datetime

print("🧪 Running configuration test...", flush=True)
print("=" * 60, flush=True)

# Select first model for testing
test_model = models_to_evaluate[0]
print(f"\n📋 Test Model: {test_model}", flush=True)

# Create a simple test dataset with one example
test_example = {
    "query": "What is the capital of France?",
    "ground_truth": "Paris",
    "response": "Paris"
}

# Save test example to a temporary file
import tempfile
test_file = tempfile.NamedTemporaryFile(mode='w', suffix='.jsonl', delete=False)
test_file.write(json.dumps(test_example) + '\n')
test_file.close()

print(f"📝 Test Query: {test_example['query']}", flush=True)

try:
    # Create target function for test model
    test_target_fn = create_target_function(test_model)
    
    # Test the target function
    print("\n1️⃣ Testing target function...", flush=True)
    sys.stdout.flush()
    
    test_result = test_target_fn(**test_example)
    print(f"   ✅ Target function returned: {test_result['response'][:100]}...", flush=True)
    
    # Test evaluation with minimal evaluators AND portal publishing
    print("\n2️⃣ Testing evaluation pipeline with portal publishing...", flush=True)
    sys.stdout.flush()
    
    from azure.ai.evaluation import evaluate
    
    test_timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    test_eval_result = evaluate(
        data=test_file.name,
        target=test_target_fn,
        evaluators={
            "relevance": relevance_eval,
            "coherence": coherence_eval,
        },
        evaluator_config={
            "default": {
                "query": "${data.query}",
                "response": "${target.response}",
            }
        },
        # Publish to portal for verification (using URL format)
        azure_ai_project=azure_ai_project_url,
        evaluation_name=f"22-evaluate-models-TEST_{test_model}_{test_timestamp}"
    )
    
    print(f"\n   ✅ Evaluation completed successfully!", flush=True)
    print(f"   📊 Sample metrics:", flush=True)
    print(f"      - Relevance: {test_eval_result['metrics'].get('relevance', 'N/A')}", flush=True)
    print(f"      - Coherence: {test_eval_result['metrics'].get('coherence', 'N/A')}", flush=True)
    
    # Show portal URL if available
    if test_eval_result.get('studio_url'):
        print(f"\n   🌐 View test results in portal:", flush=True)
        print(f"      {test_eval_result['studio_url']}", flush=True)
    
    print("\n" + "=" * 60, flush=True)
    print("✅ Configuration test PASSED! Ready to run full evaluation.", flush=True)
    print("=" * 60, flush=True)
    sys.stdout.flush()
    
except Exception as e:
    print(f"\n❌ Configuration test FAILED!", flush=True)
    print(f"Error: {str(e)}", flush=True)
    print("\nPlease fix the configuration before proceeding to Step 8.", flush=True)
    print("=" * 60, flush=True)
    sys.stdout.flush()
    raise
finally:
    # Clean up temporary file
    import os
    if os.path.exists(test_file.name):
        os.unlink(test_file.name)
    print("\n🧹 Temporary test file cleaned up.", flush=True)


## Step 8: Run Evaluations

Now we'll evaluate each model using the test dataset and the [`evaluate()` function](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/evaluate-sdk#local-evaluation-on-test-datasets-using-evaluate). This will generate comprehensive metrics for performance, quality, and safety.

Each evaluation:
- Tests the model with all examples from the dataset
- Calculates quality metrics using a judge model
- Assesses safety using Azure AI Content Safety
- Tracks evaluation time (as a proxy for latency)
- **Publishes results to Azure AI Foundry portal** for visualization
- **Saves detailed results locally** for offline analysis

In [ ]:
from azure.ai.evaluation import evaluate
import time
from datetime import datetime
import os

# Create output directory for evaluation results
output_dir = "./22-evaluate-models-results"
os.makedirs(output_dir, exist_ok=True)

# Store results for each model
evaluation_results = {}

print(f"🚀 Starting evaluation of {len(models_to_evaluate)} models...")
print(f"   Test dataset size: {len(test_data)} examples")
print(f"   Output directory: {output_dir}\n")

for model_name in models_to_evaluate:
    print(f"📊 Evaluating model: {model_name}")
    start_time = time.time()
    
    try:
        # Create target function for this model
        target_fn = create_target_function(model_name)
        
        # Create output path for this evaluation
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        output_path = os.path.join(output_dir, f"22-evaluate-models_{model_name}_{timestamp}")
        
        # Run evaluation with both portal publishing and local output
        result = evaluate(
            data=dataset_path,
            target=target_fn,
            evaluators={
                "relevance": relevance_eval,
                "coherence": coherence_eval,
                "fluency": fluency_eval,
                "violence": violence_eval,
                "hate_unfairness": hate_unfairness_eval,
                "self_harm": self_harm_eval,
                "sexual": sexual_eval,
            },
            evaluator_config={
                "default": {
                    "query": "${data.query}",
                    "response": "${target.response}",
                }
            },
            # Publish to Azure AI Foundry portal for visualization (using URL format)
            azure_ai_project=azure_ai_project_url,
            # Save detailed results locally
            output_path=output_path,
            # Optional: provide evaluation name for easier tracking in portal
            evaluation_name=f"22-evaluate-models_{model_name}_{timestamp}"
        )
        
        elapsed_time = time.time() - start_time
        
        # Store results with both portal and local file information
        evaluation_results[model_name] = {
            "metrics": result["metrics"],
            "evaluation_time": elapsed_time,
            "studio_url": result.get("studio_url"),
            "output_path": output_path
        }
        
        print(f"   ✅ Completed in {elapsed_time:.2f} seconds")
        print(f"   📊 Portal URL: {result.get('studio_url', 'N/A')}")
        print(f"   💾 Local results: {output_path}\n")
        
    except Exception as e:
        print(f"   ❌ Error evaluating {model_name}: {str(e)}\n")
        evaluation_results[model_name] = {
            "error": str(e),
            "evaluation_time": time.time() - start_time
        }

print("✅ All evaluations complete!")
print(f"\n📁 All results saved to: {output_dir}/")
print(f"🌐 View results in Azure AI Foundry portal using the URLs above")


## Step 9: Analyze Results

Let's create a summary comparison of all models across key metrics.

In [ ]:
import pandas as pd

# Prepare data for comparison
comparison_data = []

for model_name, results in evaluation_results.items():
    if "error" in results:
        print(f"⚠️  {model_name}: Evaluation failed - {results['error']}\n")
        continue
    
    metrics = results["metrics"]
    
    # The metrics are stored with keys like "relevance.relevance", "coherence.coherence", etc.
    row = {
        "Model": model_name,
        "Eval Time (s)": results['evaluation_time'],
        "Relevance": metrics.get('relevance.relevance', metrics.get('relevance', 0)),
        "Coherence": metrics.get('coherence.coherence', metrics.get('coherence', 0)),
        "Fluency": metrics.get('fluency.fluency', metrics.get('fluency', 0)),
        "Violence": metrics.get('violence.violence_defect_rate', metrics.get('violence', 0)),
        "Hate/Unfairness": metrics.get('hate_unfairness.hate_unfairness_defect_rate', metrics.get('hate_unfairness', 0)),
        "Self-Harm": metrics.get('self_harm.self_harm_defect_rate', metrics.get('self_harm', 0)),
        "Sexual": metrics.get('sexual.sexual_defect_rate', metrics.get('sexual', 0)),
    }
    
    comparison_data.append(row)

# Create comparison DataFrame
df_comparison = pd.DataFrame(comparison_data)

print("📊 Model Evaluation Comparison\n")
display(df_comparison)


## Step 10: Performance Summary

Analyze the evaluation results to identify the best performing models across different metrics.

In [ ]:
from IPython.display import display, Markdown, HTML
import pandas as pd

# Check for successful evaluations
successful_models = {name: results for name, results in evaluation_results.items() if 'error' not in results}
failed_models = {name: results for name, results in evaluation_results.items() if 'error' in results}

if successful_models:
    display(Markdown("## 🏆 Best Performing Models by Metric"))
    
    # Define evaluator metrics and their optimization direction
    evaluator_metrics = [
        ('relevance.relevance', 'Relevance', True),
        ('coherence.coherence', 'Coherence', True),
        ('fluency.fluency', 'Fluency', True),
        ('violence.violence_defect_rate', 'Violence Safety', False),
        ('hate_unfairness.hate_unfairness_defect_rate', 'Hate/Unfairness Safety', False),
        ('self_harm.self_harm_defect_rate', 'Self-Harm Safety', False),
        ('sexual.sexual_defect_rate', 'Sexual Safety', False),
    ]
    
    # Create a dataframe for best models
    best_models_data = []
    
    for metric_key, display_name, higher_is_better in evaluator_metrics:
        valid_models = {}
        
        # Collect scores for this metric from all successful models
        for model_name, results in successful_models.items():
            metrics = results['metrics']
            score = metrics.get(metric_key)
            if score is not None:
                valid_models[model_name] = score
        
        if valid_models:
            # Find best model based on optimization direction
            if higher_is_better:
                best_model_name = max(valid_models, key=valid_models.get)
                best_score = valid_models[best_model_name]
                direction = "↑ Higher is Better"
            else:
                best_model_name = min(valid_models, key=valid_models.get)
                best_score = valid_models[best_model_name]
                direction = "↓ Lower is Better"
            
            best_models_data.append({
                "Metric": display_name,
                "Best Model": best_model_name,
                "Score": f"{best_score:.3f}",
                "Direction": direction
            })
    
    df_best = pd.DataFrame(best_models_data)
    display(df_best)
    
    # Calculate overall best model (based on quality metrics average)
    display(Markdown("---"))
    display(Markdown("## 🌟 Overall Best Model (Quality Metrics Average)"))
    
    quality_metric_keys = ['relevance.relevance', 'coherence.coherence', 'fluency.fluency']
    model_quality_scores = {}
    
    for model_name, results in successful_models.items():
        metrics = results['metrics']
        scores = []
        for metric_key in quality_metric_keys:
            score = metrics.get(metric_key)
            if score is not None:
                scores.append(score)
        
        if scores:
            avg_score = sum(scores) / len(scores)
            model_quality_scores[model_name] = {
                'avg_quality': avg_score,
                'eval_time': results['evaluation_time']
            }
    
    if model_quality_scores:
        best_overall = max(model_quality_scores, key=lambda x: model_quality_scores[x]['avg_quality'])
        best_data = model_quality_scores[best_overall]
        
        display(Markdown(f"🥇 **{best_overall}** - Quality: {best_data['avg_quality']:.3f} | Time: {best_data['eval_time']:.2f}s"))
        
        # Show all model quality scores for comparison
        display(Markdown(""))
        
        ranking_data = []
        sorted_models = sorted(model_quality_scores.items(), key=lambda x: x[1]['avg_quality'], reverse=True)
        
        for rank, (model_name, data) in enumerate(sorted_models, 1):
            medal = "🥇" if rank == 1 else "🥈" if rank == 2 else "🥉" if rank == 3 else ""
            ranking_data.append({
                "Rank": f"{medal} {rank}",
                "Model": model_name,
                "Avg Quality Score": f"{data['avg_quality']:.3f}",
                "Eval Time (s)": f"{data['eval_time']:.2f}"
            })
        
        df_ranking = pd.DataFrame(ranking_data)
        display(df_ranking)

## Step 11: Next Steps

You've successfully evaluated multiple models! Here are some next steps to consider:

### 📊 View Results in Two Places

- **Azure AI Foundry Portal**: Interactive visualizations with [detailed charts and comparisons](https://learn.microsoft.com/azure/ai-foundry/how-to/evaluate-results)
- **Portal URLs**: Each evaluation includes a studio URL for easy access and team sharing
- **Local Files**: All results saved in `./22-evaluate-models-results/` for offline analysis
- **Version Control**: Commit JSON files for reproducibility and tracking over time

### 🏆 Use Model Leaderboards for Selection

- **Browse Leaderboards**: Compare models by [Quality, Safety, Cost, and Performance](https://learn.microsoft.com/azure/ai-foundry/how-to/benchmark-model-in-catalog)
- **Trade-off Analysis**: View quality vs. cost, quality vs. safety charts
- **Scenario Filtering**: Find models best suited for your use case (Q&A, coding, reasoning)
- **Access Portal**: [Azure AI Foundry Model Catalog → Browse Leaderboards](https://aka.ms/model-leaderboards)

---

**Great work! You now have comprehensive evaluation metrics for multiple models.** 🎉
